In [6]:
# ── run this ONCE at the top of every demo notebook ────────────────────
import sys, pathlib, os

# 1) current working dir where the kernel started
here = pathlib.Path(os.getcwd()).resolve()

# 2) walk upward until we find the "src" folder
while here != here.root and not (here / "src").exists():
    here = here.parent

# 3) append that folder to sys.path  (only once)
if str(here) not in sys.path:
    sys.path.append(str(here))

print("✓ added to PYTHONPATH:", here)
# ───────────────────────────────────────────────────────────────────────


✓ added to PYTHONPATH: C:\Users\tgati\molder-rl


In [8]:
# 03_xgb_demo.ipynb
# Demo: XGBoost quality alarm on 30 random test shots

import pathlib as pl
import numpy as np
import pandas as pd
import joblib
from src.models.xgb_quality_wrapper import predict_fail, prob_fail


In [10]:
TEST_PARQ = pl.Path(r"C:/Users/tgati/molder-rl/data/interim/features_test.parquet")
df_test   = pd.read_parquet(TEST_PARQ)

np.random.seed(42)
demo_idx  = np.random.choice(df_test.index, 30, replace=False)
demo_df   = df_test.loc[demo_idx].copy().reset_index(drop=True)

# keep the ground-truth label for comparison
gt_labels = demo_df.pop("label").values
demo_df.head()


,cycle_time,filling_time,plasticizing_time,clamp_close_time,max_inj_pressure,max_back_pressure,avg_back_pressure,max_screw_rpm,avg_screw_rpm,max_inj_speed,cushion_error,switch_over_error,barrel_temp_avg,hopper_temp,mold_temp_avg,delta_mold_temp,outlier_flag
0,61.779999,0.93,12.910000,6.81,142.399994,55.500000,60.500000,30.900000,290.399994,128.399994,649.270020,-15.0,232.942858,65.500000,3.900000,24.000000,1
1,59.459999,4.39,16.799999,7.12,141.800003,37.500000,59.299999,30.700001,29.200001,56.000000,648.440002,-15.0,226.185713,67.099998,3.541667,21.799999,0
2,59.480000,4.39,16.820000,7.13,141.699997,37.400002,59.200001,30.600000,29.200001,56.000000,648.440002,-15.0,225.914287,66.599998,3.591667,22.100000,0
3,59.439999,4.38,16.820000,7.12,141.699997,37.099998,59.099998,30.400000,29.200001,56.200001,648.440002,-15.0,225.985718,66.300003,3.733333,23.000000,1
4,59.520000,4.46,16.889999,7.13,142.000000,38.099998,59.599998,30.700001,29.200001,55.299999,648.440002,-15.0,225.985711,68.000000,3.900000,24.100000,0


In [11]:
preds = []
probs = []

for row in demo_df.to_numpy(dtype=np.float32):
    p = prob_fail(row)
    probs.append(p)
    preds.append(predict_fail(row))

demo_df["P_fail"]   = np.round(probs, 3)
demo_df["Pred_NG"]  = preds          # 1 = NG alarm
demo_df["True_NG"]  = gt_labels


In [12]:
def color_row(row):
    if row["True_NG"] == 1:
        return ['background-color: gold'] * len(row)  # actual defect
    if row["Pred_NG"] == 1:
        return ['background-color: salmon'] * len(row)  # false alarm
    return [''] * len(row)   # OK shot

styled = (demo_df[["P_fail", "Pred_NG", "True_NG"]]
          .style
          .apply(color_row, axis=1)
          .format({"P_fail": "{:.3f}"}))
styled


,P_fail,Pred_NG,True_NG
0,0.073,1,0
1,0.073,1,0
2,0.073,1,0
3,0.073,1,0
4,0.073,1,0
5,0.073,1,0
6,0.073,1,0
7,0.073,1,0
8,0.073,1,0
9,0.073,1,0


In [13]:
TP = ((demo_df["Pred_NG"] == 1) & (demo_df["True_NG"] == 1)).sum()
FP = ((demo_df["Pred_NG"] == 1) & (demo_df["True_NG"] == 0)).sum()
FN = ((demo_df["Pred_NG"] == 0) & (demo_df["True_NG"] == 1)).sum()

print(f"Caught defects  (TP): {TP}")
print(f"False alarms   (FP): {FP}")
print(f"Missed defects (FN): {FN}")


Caught defects  (TP): 0
False alarms   (FP): 30
Missed defects (FN): 0
